In [1]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

import os

from PIL import Image

Using TensorFlow backend.


In [2]:
def load_data():
        data = []
        small = []
        paths = []
        #get all files in this folder
        for r, d, f in os.walk(r"D:\Downloads\selfie2anime\trainB"):
            for file in f:
                if '.jpg' in file:
                    paths.append(os.path.join(r, file))
        #for each file add normal resolution and low resolution to arrays
        for path in paths:
            img = Image.open(path)
            x = np.array(img.resize((64,64)))
            y = np.array(img)
            #uncomment to remove alpha channel for PNGs
            #x = x[...,:3]
            data.append(y)
            small.append(x)
            
        #reshaping data to be four dimension required for input to neural network
        y_train = np.array(data)
        y_train = y_train.reshape(len(data),256,256,3)
        x_train = np.array(small)
        x_train = x_train.reshape(len(small),64,64,3)
        return y_train, x_train

In [3]:
class SRGAN():
    def __init__(self):
        
        # Shape of high resolution output image
        self.img_rows = 256
        self.img_cols = 256
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        # Shape of low resolution input image
        self.latent_dim = (64,64,3)

        #optimizer (learning rate and beta values)
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()
        generator = self.generator

        # The generator takes noise as input and generates imgs
        z = Input(shape=self.latent_dim)
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()
        model.add(Conv2D(120, input_shape=self.latent_dim, kernel_size=(3,3), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(120, kernel_size=(3,3), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(UpSampling2D())
        model.add(Conv2D(120, kernel_size=(2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(120, kernel_size=(4,4), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(UpSampling2D())
        model.add(Conv2D(120, kernel_size=(3,3), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(120, kernel_size=(3,3), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(3, kernel_size=(2,2), padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.summary()

        noise = Input(shape=self.latent_dim)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        Y_train, X_train = load_data()

        # Rescale to be between 0 & 1
        X_train = X_train / 255
        Y_train = Y_train / 255

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Placeholder for loss function values
        g_loss_epochs = np.zeros((epochs, 1))
        d_loss_epochs = np.zeros((epochs, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, Y_train.shape[0], batch_size)
            imgs = Y_train[idx]

            # Generate super resolution images from the random batch of images
            gen_imgs = self.generator.predict(X_train[idx])

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(X_train[idx], valid)
            
            #save loss history
            g_loss_epochs[epoch] = g_loss
            d_loss_epochs[epoch] = d_loss[0]

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch, X_train, idx)
            
        return g_loss_epochs, d_loss_epochs

    def save_imgs(self, epoch, X_train, idx):
        r, c = 3, 3
        # Select 9 random images
        index = np.random.randint(0, X_train.shape[0], 100)
        images = X_train[idx]
        # Super resolution the images
        gen_imgs = self.generator.predict(images)
        gen_imgs = np.array(gen_imgs) * 255
        gen_imgs = gen_imgs.astype(int)
        # Plot each image
        fig=plt.figure(figsize=(20, 20))
        for i in range(1, c*r+1):
            img = gen_imgs[i-1]
            fig.add_subplot(r, c, i)
            plt.imshow(img)
        fig.savefig(r"C:\Users\Vee\Desktop\python\GAN\epoch_%d.png" % epoch)
        plt.close()
        # save model to .h5 file
        self.generator.save("generator.h5")

In [4]:
gan = SRGAN()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 128, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 64)        256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 64, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 64, 64)       

In [ ]:
g_loss, d_loss = gan.train(epochs=20001, batch_size=10, save_interval=100)

C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0

0 [D loss: 3.784798, acc.: 15.00%] [G loss: 0.073473]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 1.165950, acc.: 50.00%] [G loss: 3.827034]
2 [D loss: 2.350181, acc.: 40.00%] [G loss: 1.334147]
3 [D loss: 1.926417, acc.: 5.00%] [G loss: 0.867204]
4 [D loss: 1.202445, acc.: 30.00%] [G loss: 0.693736]
5 [D loss: 1.226032, acc.: 35.00%] [G loss: 0.102739]
6 [D loss: 1.362086, acc.: 30.00%] [G loss: 0.374709]
7 [D loss: 0.937363, acc.: 60.00%] [G loss: 0.149160]
8 [D loss: 0.657709, acc.: 75.00%] [G loss: 0.145050]
9 [D loss: 0.179395, acc.: 95.00%] [G loss: 0.207185]
10 [D loss: 0.200716, acc.: 90.00%] [G loss: 0.230032]
11 [D loss: 0.327591, acc.: 85.00%] [G loss: 0.672062]
12 [D loss: 0.376309, acc.: 80.00%] [G loss: 1.212704]
13 [D loss: 1.469352, acc.: 20.00%] [G loss: 1.205878]
14 [D loss: 0.888557, acc.: 60.00%] [G loss: 2.128604]
15 [D loss: 1.025703, acc.: 30.00%] [G loss: 1.888548]
16 [D loss: 0.838889, acc.: 50.00%] [G loss: 5.226865]
17 [D loss: 2.169756, acc.: 5.00%] [G loss: 2.930492]
18 [D loss: 2.288221, acc.: 10.00%] [G loss: 2.844561]
19 [D loss: 1.572456,

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


...

901 [D loss: 1.011087, acc.: 45.00%] [G loss: 1.915667]
902 [D loss: 0.686059, acc.: 50.00%] [G loss: 1.458012]
903 [D loss: 1.320444, acc.: 55.00%] [G loss: 2.197258]
904 [D loss: 0.991912, acc.: 55.00%] [G loss: 3.732198]
905 [D loss: 1.981916, acc.: 10.00%] [G loss: 1.815904]
906 [D loss: 1.438556, acc.: 40.00%] [G loss: 1.581141]
907 [D loss: 1.219663, acc.: 50.00%] [G loss: 1.993969]
908 [D loss: 1.351426, acc.: 35.00%] [G loss: 2.793697]
909 [D loss: 0.983034, acc.: 65.00%] [G loss: 0.934728]
910 [D loss: 1.489400, acc.: 40.00%] [G loss: 1.767362]
911 [D loss: 0.946893, acc.: 50.00%] [G loss: 1.798622]
912 [D loss: 0.642642, acc.: 70.00%] [G loss: 2.094504]
913 [D loss: 1.252572, acc.: 45.00%] [G loss: 1.866804]
914 [D loss: 0.928455, acc.: 45.00%] [G loss: 2.710680]
915 [D loss: 0.745554, acc.: 70.00%] [G loss: 2.744526]
916 [D loss: 1.210119, acc.: 35.00%] [G loss: 1.344050]
917 [D loss: 1.251310, acc.: 40.00%] [G loss: 1.919237]
918 [D loss: 0.846700, acc.: 60.00%] [G loss: 2.

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


1000 [D loss: 0.889749, acc.: 55.00%] [G loss: 0.716815]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


1001 [D loss: 0.880612, acc.: 60.00%] [G loss: 1.260074]
1002 [D loss: 0.567708, acc.: 65.00%] [G loss: 1.734889]
1003 [D loss: 1.357617, acc.: 45.00%] [G loss: 1.515663]
1004 [D loss: 0.899822, acc.: 45.00%] [G loss: 1.521122]
1005 [D loss: 0.922614, acc.: 55.00%] [G loss: 1.451353]
1006 [D loss: 1.195934, acc.: 45.00%] [G loss: 1.836034]
1007 [D loss: 0.910982, acc.: 60.00%] [G loss: 1.569072]
1008 [D loss: 1.142583, acc.: 55.00%] [G loss: 2.185969]
1009 [D loss: 0.896961, acc.: 65.00%] [G loss: 2.902854]
1010 [D loss: 0.681356, acc.: 75.00%] [G loss: 2.201629]
1011 [D loss: 0.825907, acc.: 55.00%] [G loss: 2.799598]
1012 [D loss: 1.088745, acc.: 45.00%] [G loss: 0.940833]
1013 [D loss: 1.010046, acc.: 50.00%] [G loss: 1.842981]
1014 [D loss: 0.501712, acc.: 75.00%] [G loss: 1.399047]
1015 [D loss: 0.761987, acc.: 70.00%] [G loss: 3.806377]


In [ ]:
plt.plot(g_loss)
plt.plot(d_loss)
plt.title('GAN Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Generator', 'Discriminator'], loc='upper left')
plt.show()